In [3]:
import numpy as np
import csv
import re
import os

In [4]:
our_data_dir = 'our_data'
our_output_dir = 'our_output'
data_dir = 'data'

feature_names = os.listdir(our_data_dir)

feature_names

['jurkat.csv', 'matek.csv', 'retinopathy.csv', 'isic.csv', 'cifar10.csv']

In [8]:
def generate_output(features_name):
    dataset = re.search(r'(.+)\.csv', features_name).group(1)

    names = []
    path = os.path.join(our_data_dir, features_name)
    with open(path, 'r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            names.append(row['name'])

    # indices
    indices = []
    path = f'output/results_data_processed_our_{dataset}.pickle_tsne_kmediods_csd_100_0/X100-0-indicies.csv'
    with open(path, 'r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            indices.append(int(row['indices']))

    print(f'indices: {indices}')

    # names
    path = os.path.join(our_output_dir, features_name)
    with open(path, 'w') as csv_file:
        csv_writer = csv.writer(csv_file)

        csv_writer.writerow(['name'])
        for i in indices:
            csv_writer.writerow([names[i]])


In [9]:
for feature_name in feature_names:
    generate_output(feature_name)

indices: [13272, 15016, 14970, 10174, 12027, 23509, 24339, 5776, 4405, 18487, 5601, 16238, 22711, 15053, 23803, 20115, 22333, 25394, 3703, 25026, 2630, 13484, 23313, 6207, 10324, 3803, 18257, 7064, 4211, 22937, 11885, 16555, 10620, 13358, 6479, 10572, 20044, 25392, 8050, 1319, 5192, 5267, 6961, 21438, 6363, 1077, 23297, 14164, 13635, 17853, 17236, 25506, 20445, 3899, 18735, 23412, 1371, 9868, 18151, 20009, 18302, 9110, 1948, 248, 23182, 24632, 8138, 21379, 7206, 10510, 1346, 4747, 11210, 3755, 25291, 12471, 3918, 14113, 22413, 21249, 8532, 11547, 20550, 18300, 16268, 11959, 15252, 10425, 5315, 8539, 7917, 8058, 17550, 21803, 6097, 13660, 2155, 13421, 4740, 14224]
indices: [2674, 9255, 5422, 12622, 13262, 697, 11180, 6457, 4936, 6332, 9484, 9401, 5993, 4089, 2630, 681, 10201, 12730, 7302, 11902, 3461, 7708, 10320, 427, 410, 11416, 14442, 13197, 4394, 4096, 9674, 6748, 9865, 6765, 1192, 11296, 12346, 4701, 12259, 14247, 1029, 4706, 2012, 11439, 5241, 1755, 7041, 6533, 10478, 13282, 12377